In [1]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np

from IPython import get_ipython
from IPython.display import display


# Check for CUDA availability and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Heston model parameters
r = 0.1      # risk-free rate
sigma = 0.25    # initial variance

# Domain boundaries
x_min, x_max = 0.1, 3.0        # moneyness range [0.1, 3.0]
T = 1.0                       # maturity (1 year)
N_steps = 100                 # number of time steps
dt = T / N_steps              # time step size

# Training settings
train_stages = 2000           # training iterations per time step
M_samples = 1800              # samples per training iteration (600*(d+1) with d=2)


Using device: cuda


In [2]:
import math

class ResidualLayer(torch.nn.Module):
    def __init__(self, dim):
        super(ResidualLayer, self).__init__()
        self.linear = torch.nn.Linear(dim, dim)
        torch.nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, x):
        h = torch.tanh(self.linear(x))
        return h + x

class ResidualNet(torch.nn.Module):
    def __init__(self, layer_width, input_dim, r=0.0, t=0.0, CP=1, xp=2.0):
        super(ResidualNet, self).__init__()
        self.xp = xp
        self.r = r
        self.t = t
        self.CP = CP
        self.W = torch.nn.Linear(input_dim, layer_width)
        torch.nn.init.xavier_uniform_(self.W.weight)
        self.res1 = ResidualLayer(layer_width)
        self.res2 = ResidualLayer(layer_width)
        self.res3 = ResidualLayer(layer_width)
        self.W4 = torch.nn.Linear(layer_width, 1, bias=False)
        self.W4.weight = torch.nn.Parameter(torch.ones((1, layer_width)))

    def forward_main(self, X):
        payoff = torch.max(torch.tensor(0.0).to('cuda'), self.CP * (self.S - math.exp(-self.r * self.t)))
        if self.t == 0.0:
            return payoff
        else:
            X1 = torch.tanh(self.W(X))
            X2 = self.res1(X1)
            X3 = self.res2(X2)
            X4 = self.res3(X3)
            return payoff + torch.log(torch.exp(self.W4(X4)) + 1)

    def forward(self, *args):
        x = torch.concat([*args], 1)
        self.S = x[:, 0, None]
        if self.CP == 1:
            S_box = torch.where(self.S > self.xp, self.xp * torch.ones_like(self.S), self.S).requires_grad_()
            S_diff = (self.S - S_box).requires_grad_()
            x[:, 0, None] = S_box
            return self.forward_main(x) + S_diff
        else:
            return self.forward_main(x)


In [3]:
# Option parameters
T = 1.0
r = 0.05
sigma = 0.25
option = 'call'
CP = 1 if option == 'call' else -1
name = 'BS_' + option
xp = 2.0

# Neural network parameters
nodes_per_layer = 50
d = 1  # Input dimension: S

# Sampling parameters
nSim_dim = 600
nSim_TDGF = nSim_dim * d
S_low = 0.01
S_high = 3.0
Omega = S_high - S_low

# TDGF parameters
N_t = 100
h = T / N_t

# Training parameters
sampling_stages_DNM = 2000

# Terminal payoff function
def Phi(x):
    return torch.max(torch.tensor(0.0).to('cuda'), CP * (x - 1.0))

# Sampling function
def sampler(nSim):
    t = T * torch.rand([nSim, 1])
    S = S_low + torch.rand([nSim, 1]) * Omega
    return t.requires_grad_().to('cuda'), S.requires_grad_().to('cuda')

# Loss function for pretraining (terminal condition)
def lossTerminal(S_int):
    f = DNM_model(S_int)
    return torch.mean((f - Phi(S_int))**2)

# Loss function for TDGF
def lossDNM(S_int):
    f = DNM_model(S_int)
    f_S = torch.autograd.grad(f, S_int, grad_outputs=torch.ones_like(f), create_graph=True)[0]
    a = 0.5 * sigma**2 * S_int**2
    Lagrangian = 0.5 * (a * f_S**2 + r * f**2)
    G1 = torch.mean(Lagrangian)
    f_old = old_model(S_int)
    old_S = torch.autograd.grad(f_old, S_int, grad_outputs=torch.ones_like(f), create_graph=True)[0]
    b = (sigma**2 - r) * S_int
    F = b * old_S
    G2 = torch.mean(F * f)
    G3 = torch.mean((f - f_old)**2)
    return 0.5 * G3 + h * (G1 + G2)

In [4]:
import scipy.stats as st

# Black-Scholes analytical solution
def black_scholes_price(S, K, t, T, r, sigma, option_type):
    """
    Calculates the Black-Scholes option price.

    Args:
        S (float or torch.Tensor): Underlying price.
        K (float): Strike price.
        t (float): Current time.
        T (float): Maturity time.
        r (float): Risk-free rate.
        sigma (float): Volatility.
        option_type (str): 'call' or 'put'.

    Returns:
        float or torch.Tensor: Black-Scholes option price.
    """
    # Ensure S is a tensor for calculations
    if not isinstance(S, torch.Tensor):
        S = torch.tensor(S, dtype=torch.float32, device=device)

    # Ensure T-t is a tensor for torch.sqrt
    time_to_maturity = T - t
    if not isinstance(time_to_maturity, torch.Tensor):
        time_to_maturity = torch.tensor(time_to_maturity, dtype=torch.float32, device=S.device) # Use same device as S

    # Handle potential division by zero if time_to_maturity is very small or zero
    # Add a small epsilon to avoid issues if time_to_maturity is exactly zero
    epsilon = 1e-9
    sqrt_time_to_maturity = torch.sqrt(time_to_maturity + epsilon)

    d1 = (torch.log(S / K) + (r + 0.5 * sigma**2) * time_to_maturity) / (sigma * sqrt_time_to_maturity)
    d2 = d1 - sigma * sqrt_time_to_maturity

    # Use torch.distributions.normal.cdf for tensor-compatible CDF
    # Need to import torch.distributions
    from torch.distributions import Normal
    normal_dist = Normal(torch.tensor([0.0], device=S.device), torch.tensor([1.0], device=S.device))

    if option_type == 'call':
        price = S * normal_dist.cdf(d1) - K * torch.exp(-r * time_to_maturity) * normal_dist.cdf(d2)
    elif option_type == 'put':
        price = K * torch.exp(-r * time_to_maturity) * normal_dist.cdf(-d2) - S * normal_dist.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    return price

In [5]:
import os
import time
import matplotlib.pyplot as plt # Import matplotlib for plotting

# Ensure necessary classes and functions are available (assuming they are defined in previous cells)
# from . import DGMNet, sampler, lossTerminal, lossDNM, device, name, h, T, N_t, nSim_TDGF, sampling_stages_DNM, S_low, S_high

# Train TDGF network
start_time = time.time()

# Assuming DNM_model is already defined and initialized
# For demonstration, let's initialize it if it's not already
try:
    DNM_model
except NameError:
    # These parameters should match your model initialization
    nodes_per_layer = 50
    d = 1
    r = 0.05
    T = 1.0 # Make sure T is defined
    CP = 1 # Make sure CP is defined
    xp = 2.0 # Make sure xp is defined
    DNM_model = ResidualNet(nodes_per_layer, d, r=r, t=0.0, CP=CP, xp=xp).to(device)


# Pretraining to approximate terminal condition
print("Starting pretraining...")
optimizer = torch.optim.Adam(DNM_model.parameters(), 3e-4)
# For pretraining, we need samples at time T. The current sampler provides t from [0,T].
# Let's assume we only use the S values and conceptually consider them as S at time T.
# If more accurate sampling at T is needed, modify the sampler for the pretraining phase.
_, S_pretrain = sampler(sampling_stages_DNM * nSim_TDGF) # Sample a large dataset for pretraining
S_pretrain = S_pretrain.to(device).requires_grad_() # Ensure on device and requires grad

pretrain_batch_size = nSim_TDGF
num_pretrain_batches = (sampling_stages_DNM * nSim_TDGF) // pretrain_batch_size

for j in range(num_pretrain_batches):
    start_idx = j * pretrain_batch_size
    end_idx = (j + 1) * pretrain_batch_size
    S_batch = S_pretrain[start_idx:end_idx]

    optimizer.zero_grad()
    loss = lossTerminal(S_batch) # Make sure lossTerminal is defined
    loss.backward()
    optimizer.step()
    if (j + 1) % 100 == 0: # Print loss every 100 batches
        print(f"Pretraining batch {j+1}/{num_pretrain_batches}, Loss: {loss.item():.6f}")

# Handle remaining pretraining samples
if (sampling_stages_DNM * nSim_TDGF) % pretrain_batch_size != 0:
    start_idx = num_pretrain_batches * pretrain_batch_size
    S_batch = S_pretrain[start_idx:]
    optimizer.zero_grad()
    loss = lossTerminal(S_batch)
    loss.backward()
    optimizer.step()
    print(f"Pretraining final batch, Loss: {loss.item():.6f}")

print("Pretraining finished.")

# Create the 'weights' directory if it doesn't exist
os.makedirs('weights', exist_ok=True)
os.makedirs('plots', exist_ok=True) # Create 'plots' directory

# Save the pretrained model (which is at t=0.0)
filename_initial = 'weights/' + name + '_t=0.0' # Make sure name is defined
# Set the model time to 0.0 before saving the pretrained model
DNM_model.t = 0.0
# torch.save(DNM_model.state_dict(), filename_initial) # Save state_dict - Keep this line
torch.save(DNM_model.state_dict(), filename_initial) # Save state_dict

# Load the pretrained model as the initial old_model for the first time step training
# Instead of loading the OrderedDict, instantiate a new model and load the state_dict
old_model = ResidualNet(nodes_per_layer, d, r=r, t=0.0, CP=CP, xp=xp).to(device) # Instantiate a new model and move it to the device
# Load the state dictionary into the newly created model instance
old_model.load_state_dict(torch.load(filename_initial, weights_only=False))

# Ensure old_model's time parameter is set to the time corresponding to the saved model
old_model.t = 0.0

# Ensure K (Strike price) is defined for the Black-Scholes calculation
# Based on the Phi function `CP * (x - 1.0)`, the strike price K appears to be 1.0.
K = 1.0

# Train over time steps
print("\nStarting time step training...")
# Make sure h and N_t are defined
for i, curr_t_tensor in enumerate(torch.linspace(h, T, N_t)):
    curr_t = curr_t_tensor.item() # Get the scalar value of the current time
    print(f"\nTraining for time step {i+1}/{N_t}, t = {curr_t:.4f}")

    # Set the current time in the model being trained
    DNM_model.t = curr_t

    # --- Sample the whole dataset for this time step ---
    # Total samples for this time step training
    total_samples_for_timestep = sampling_stages_DNM * nSim_TDGF
    # Sample a large batch of S values.
    _, S_sampled_for_timestep = sampler(total_samples_for_timestep)
    # Ensure the sampled data is on the correct device and requires gradients
    S_sampled_for_timestep = S_sampled_for_timestep.to(device).requires_grad_()

    optimizer = torch.optim.Adam(DNM_model.parameters(), 3e-4)

    # --- Iterate through the pre-sampled data in batches ---
    batch_size = nSim_TDGF
    num_batches = total_samples_for_timestep // batch_size

    for j in range(num_batches):
        # Get a batch of data
        start_idx = j * batch_size
        end_idx = (j + 1) * batch_size
        S_batch = S_sampled_for_timestep[start_idx:end_idx]

        optimizer.zero_grad()
        # Pass the current batch of S to lossDNM. The lossDNM function uses the current time 'DNM_model.t'
        # and the 'old_model.t' internally.
        loss = lossDNM(S_batch) # Make sure lossDNM is defined
        loss.backward()
        optimizer.step()
        if (j + 1) % (500 // (total_samples_for_timestep // (sampling_stages_DNM * batch_size))) == 0: # Adjust printing frequency based on batches
             print(f"  Time step {i+1}, Batch {j+1}/{num_batches}, Loss: {loss.item():.6f}")

    # Handle any remaining samples if total_samples_for_timestep is not perfectly divisible by batch_size
    if total_samples_for_timestep % batch_size != 0:
        start_idx = num_batches * batch_size
        S_batch = S_sampled_for_timestep[start_idx:]
        optimizer.zero_grad()
        loss = lossDNM(S_batch)
        loss.backward()
        optimizer.step()
        print(f"  Time step {i+1}, Final batch, Loss: {loss.item():.6f}")


    # Save current model state_dict
    filename_curr = 'weights/' + name + '_t=' + str(round(curr_t, 4)) # Use 4 decimal places for time
    torch.save(DNM_model.state_dict(), filename_curr)

    # Load the just-saved model as old_model for the *next* time step
    # Instantiate a new model and load the state_dict
    old_model = ResidualNet(nodes_per_layer, d, r=r, t=curr_t, CP=CP, xp=xp).to(device) # Instantiate new model with updated time
    old_model.load_state_dict(torch.load(filename_curr, weights_only=False)) # Load the state dict into the new model

    # Plotting the option price vs S for the current time step
    print("Generating plot...")
    S_plot = torch.linspace(S_low, S_high, 100).unsqueeze(1).to(device).requires_grad_()
    with torch.no_grad(): # Disable gradient calculation for plotting
        # Set the time in the model for plotting (already set for training, but good practice)
        DNM_model.t = curr_t
        # The forward pass of DGMNet expects inputs as *args. S_plot is the first argument.
        # If your model is only taking S as input (d=1), this should work.
        # If your model needs time as an input (d=2 for t, S), the forward pass and the
        # DGMNet initialization (input_dim) would need adjustment.
        option_price = DNM_model(S_plot)

        tau = T-curr_t
        # Calculate Black-Scholes price at the current time curr_t
        bs_price = black_scholes_price(S_plot, K, tau, T, r, sigma, option)


    # Detach the tensors from the computation graph before converting to NumPy
    S_plot_np = S_plot.cpu().squeeze().detach().numpy()
    option_price_np = option_price.cpu().squeeze().detach().numpy()
    bs_price_np = bs_price.cpu().squeeze().detach().numpy()

    plt.figure(figsize=(10, 6))
    plt.plot(S_plot_np, option_price_np, label=f'DGMNet t = {curr_t:.4f}')
    plt.plot(S_plot_np, bs_price_np, label=f'Black-Scholes t = {curr_t:.4f}', linestyle='--')
    plt.xlabel('Underlying Price (S)')
    plt.ylabel('Option Price')
    plt.title(f'{name} Price vs S at t = {curr_t:.4f}')
    plt.grid(True)
    plt.legend()
    plot_filename = f'plots/{name}_t={round(curr_t, 4)}.png'
    plt.savefig(plot_filename)
    plt.close()
    print(f"Plot saved to {plot_filename}")


print("\nTime step training finished.")
end_time = time.time()
print(f"Total training time: {end_time - start_time:.2f} seconds")

Starting pretraining...
Pretraining batch 100/2000, Loss: 0.000000
Pretraining batch 200/2000, Loss: 0.000000
Pretraining batch 300/2000, Loss: 0.000000
Pretraining batch 400/2000, Loss: 0.000000
Pretraining batch 500/2000, Loss: 0.000000
Pretraining batch 600/2000, Loss: 0.000000
Pretraining batch 700/2000, Loss: 0.000000
Pretraining batch 800/2000, Loss: 0.000000
Pretraining batch 900/2000, Loss: 0.000000
Pretraining batch 1000/2000, Loss: 0.000000
Pretraining batch 1100/2000, Loss: 0.000000
Pretraining batch 1200/2000, Loss: 0.000000
Pretraining batch 1300/2000, Loss: 0.000000
Pretraining batch 1400/2000, Loss: 0.000000
Pretraining batch 1500/2000, Loss: 0.000000
Pretraining batch 1600/2000, Loss: 0.000000
Pretraining batch 1700/2000, Loss: 0.000000
Pretraining batch 1800/2000, Loss: 0.000000
Pretraining batch 1900/2000, Loss: 0.000000
Pretraining batch 2000/2000, Loss: 0.000000
Pretraining finished.

Starting time step training...

Training for time step 1/100, t = 0.0100
  Time st



```
# This is formatted as code
39 minutes



In [6]:
# Define the strike price K (assuming it's 1.0 from the Phi function)
K = 1.0

# Generate evaluation points for S
S_eval = torch.linspace(S_low, S_high, 1000).unsqueeze(1).to(device)

mse_values = []
mae_values = []
times_evaluated = []

print("\nEvaluating MSE and MAE at different time steps...")

# Select 10 time steps for evaluation
eval_indices = np.linspace(0, N_t - 1, 10, dtype=int)
# Ensure the last time step is included
if N_t - 1 not in eval_indices:
    eval_indices = np.append(eval_indices, N_t - 1)
eval_indices = np.unique(eval_indices) # Remove duplicates

all_errors = []

for i in eval_indices:
    curr_t = (i + 1) * h # Calculate the time for the current time step
    times_evaluated.append(curr_t)

    # Load the model saved at this time step
    filename_curr = 'weights/' + name + '_t=' + str(round(curr_t, 4)) # Match the saving filename format
    try:
        # Instantiate the model first
        # These parameters should match your model initialization
        # Ensure nodes_per_layer, d, r, T, CP, xp are defined and consistent
        nodes_per_layer = 50 # Example value, replace with actual value
        d = 1 # Example value, replace with actual value
        r = 0.05 # Example value, replace with actual value
        # T is already defined
        CP = 1 # Example value, replace with actual value
        xp = 2.0 # Example value, replace with actual value

        # Instantiate the model with the correct parameters and move it to the device
        loaded_model = ResidualNet(nodes_per_layer, d, r=r, t=curr_t, CP=CP, xp=xp).to(device)

        # Load the state dictionary into the instantiated model
        # Use map_location=device to ensure tensors are loaded onto the correct device
        loaded_model.load_state_dict(torch.load(filename_curr, map_location=device))

        loaded_model.eval() # Set the model to evaluation mode
        # The time parameter is already set during instantiation, but can be re-set for clarity
        loaded_model.t = curr_t

        # Get the DGMNet prediction
        with torch.no_grad():
            # The forward pass expects *args. S_eval is the first argument.
            # If your model needs time as an input (d=2 for t, S), you would need to
            # provide both t and S to the forward pass, e.g., loaded_model(t_eval, S_eval)
            # where t_eval is a tensor of current time values.
            # Given d=1, the model likely only takes S.
            dgm_price = loaded_model(S_eval)


        # Calculate the Black-Scholes price at this time step
        tau = T - curr_t
        bs_price = black_scholes_price(S_eval, K, tau, T, r, sigma, option)

        # Calculate errors
        errors = dgm_price - bs_price
        mse = torch.mean(errors**2).item()
        mae = torch.mean(torch.abs(errors)).item()

        mse_values.append(mse)
        mae_values.append(mae)
        all_errors.append(errors.cpu().numpy()) # Store errors for final calculation

        # Print with scientific notation
        print(f"  Time t = {curr_t:.4f}: MSE = {mse:.6e}, MAE = {mae:.6e}")

    except FileNotFoundError:
        print(f"  Model file not found for time t = {curr_t:.4f}. Skipping.")
    except Exception as e:
        print(f"  An error occurred for time t = {curr_t:.4f}: {e}")


# Calculate overall MSE and MAE
if all_errors:
    all_errors_combined = np.concatenate(all_errors)
    final_mse = np.mean(all_errors_combined**2)
    final_mae = np.mean(np.abs(all_errors_combined))

    # Print with scientific notation
    print(f"\nFinal Overall MSE (averaged over evaluated time steps): {final_mse:.6e}")
    print(f"Final Overall MAE (averaged over evaluated time steps): {final_mae:.6e}")
else:
    print("\nNo model files were loaded for evaluation.")


Evaluating MSE and MAE at different time steps...
  Time t = 0.0100: MSE = 8.030138e-07, MAE = 4.935141e-04
  Time t = 0.1200: MSE = 2.853910e-06, MAE = 6.380667e-04
  Time t = 0.2300: MSE = 1.573004e-06, MAE = 5.873141e-04
  Time t = 0.3400: MSE = 2.083292e-06, MAE = 8.124989e-04
  Time t = 0.4500: MSE = 2.370743e-06, MAE = 9.849584e-04
  Time t = 0.5600: MSE = 1.882911e-06, MAE = 8.385243e-04
  Time t = 0.6700: MSE = 2.760801e-06, MAE = 9.920127e-04
  Time t = 0.7800: MSE = 3.114200e-06, MAE = 1.150759e-03
  Time t = 0.8900: MSE = 2.221741e-06, MAE = 1.032430e-03
  Time t = 1.0000: MSE = 3.401763e-06, MAE = 1.275177e-03

Final Overall MSE (averaged over evaluated time steps): 2.306538e-06
Final Overall MAE (averaged over evaluated time steps): 8.805254e-04




Final Overall MSE (averaged over evaluated time steps): 2.306538e-06

Final Overall MAE (averaged over evaluated time steps): 8.805254e-04


In [7]:
import os
from google.colab import files

# Define the folders you want to download
folders_to_download = ['plots', 'sample_data', 'weights']

for folder_name in folders_to_download:
    # Check if the folder exists
    if os.path.exists(folder_name):
        # Create a zip file of the folder
        zip_filename = f'{folder_name}.zip'
        !zip -r "{zip_filename}" "{folder_name}"

        # Download the zip file
        try:
            files.download(zip_filename)
            print(f"Downloaded {zip_filename}")
        except Exception as e:
            print(f"Error downloading {zip_filename}: {e}")
    else:
        print(f"Folder '{folder_name}' not found.")

  adding: plots/ (stored 0%)
  adding: plots/BS_call_t=0.1.png (deflated 12%)
  adding: plots/BS_call_t=0.65.png (deflated 11%)
  adding: plots/BS_call_t=0.03.png (deflated 11%)
  adding: plots/BS_call_t=0.67.png (deflated 10%)
  adding: plots/BS_call_t=0.71.png (deflated 11%)
  adding: plots/BS_call_t=0.27.png (deflated 11%)
  adding: plots/BS_call_t=0.53.png (deflated 10%)
  adding: plots/BS_call_t=0.79.png (deflated 11%)
  adding: plots/BS_call_t=0.23.png (deflated 11%)
  adding: plots/BS_call_t=0.72.png (deflated 11%)
  adding: plots/BS_call_t=0.25.png (deflated 11%)
  adding: plots/BS_call_t=0.66.png (deflated 11%)
  adding: plots/BS_call_t=0.84.png (deflated 11%)
  adding: plots/BS_call_t=0.64.png (deflated 11%)
  adding: plots/BS_call_t=0.2.png (deflated 11%)
  adding: plots/BS_call_t=0.8.png (deflated 11%)
  adding: plots/BS_call_t=0.59.png (deflated 11%)
  adding: plots/BS_call_t=0.97.png (deflated 10%)
  adding: plots/BS_call_t=0.35.png (deflated 11%)
  adding: plots/BS_call_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded plots.zip
  adding: sample_data/ (stored 0%)
  adding: sample_data/README.md (deflated 39%)
  adding: sample_data/anscombe.json (deflated 83%)
  adding: sample_data/mnist_train_small.csv (deflated 88%)
  adding: sample_data/california_housing_test.csv (deflated 76%)
  adding: sample_data/california_housing_train.csv (deflated 79%)
  adding: sample_data/mnist_test.csv (deflated 88%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded sample_data.zip
  adding: weights/ (stored 0%)
  adding: weights/BS_call_t=0.87 (deflated 13%)
  adding: weights/BS_call_t=0.05 (deflated 13%)
  adding: weights/BS_call_t=0.44 (deflated 13%)
  adding: weights/BS_call_t=0.17 (deflated 13%)
  adding: weights/BS_call_t=0.09 (deflated 13%)
  adding: weights/BS_call_t=0.81 (deflated 13%)
  adding: weights/BS_call_t=0.51 (deflated 13%)
  adding: weights/BS_call_t=0.86 (deflated 13%)
  adding: weights/BS_call_t=0.89 (deflated 13%)
  adding: weights/BS_call_t=0.9 (deflated 13%)
  adding: weights/BS_call_t=0.64 (deflated 13%)
  adding: weights/BS_call_t=0.63 (deflated 13%)
  adding: weights/BS_call_t=0.01 (deflated 13%)
  adding: weights/BS_call_t=0.88 (deflated 13%)
  adding: weights/BS_call_t=0.16 (deflated 13%)
  adding: weights/BS_call_t=0.85 (deflated 13%)
  adding: weights/BS_call_t=0.07 (deflated 13%)
  adding: weights/BS_call_t=0.66 (deflated 13%)
  adding: weights/BS_call_t=0.95 (deflated 13%)
  adding: weights/BS_call_t=0.9

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded weights.zip
